In [1]:
import torch
import re
import jiwer
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_from_disk
from evaluate import load

In [2]:
device = "cuda:0"

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, low_cpu_mem_usage=True,
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=1,  # batch size for inference - set based on your device
    device=device,
)

Device set to use cuda:0


In [3]:
dss = {
    "cslu": load_from_disk("../data/cslu_kids.ds"),
}

In [4]:
# Test
sample = dss["cslu"][0]["audio"]

result = pipe(sample)
print(dss["cslu"][0]["sentence"])
print(result)

/home/jovyan/conda_envs/hf/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<bn> a b c d e f g <br> h i j k<ln> l m n o p<ln> <br> q r s t u v w x y and z <bn> <pau> my<bn> family<bn> <bn> she<bn> went<bn> to<bn> go<bn> pick<bn> up<bn> my<bn> little<bn> sister<bn> and<bn> she's gonna<bn> <br> come<bn> tomorrow she's gonna come at eleven <pau> yeah <pau> <bn> okay <bn> clean my room <bn> and<bn> then<bn> when<bn> i'm<bn> done<bn> i<bn> get<bn> to<bn> play<bn> with<bn> my<bn> friend<bn> <pau> brittney we go over to her house and we play barbies <pau> and <br> we uhm <pau> we ride our bikes after we're done and then we eat some ice cream <pau> i have four sisters <pau> <bs> one's fifteen <bn> th* four* thirteen <br> and ten and one's five <pau> yeah <pau> <bn> they're nice and they let me <br> uhm watch tv<ln> in their room <bs> and uhm <br> <pau> and<bn> <br> she<bn> when sometimes<ln> when i <br> do a little bit of chores <br> she gives me a dollar
{'text': " A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, and Z. My family. She went t

# Run on all Data

In [5]:
results = pipe(dss["cslu"]["audio"])

In [28]:
from text2digits import text2digits
import string

t2d = text2digits.Text2Digits()
punctuation_remover = str.maketrans('', '', string.punctuation)

def normalize_transcript(text):
    # The original transcript has annotations, for example a pause is <pau>
    # Remove tags in angle brackets
    text = re.sub(r'<[^>]*>', '', text)
    
    # These are "false starts" in the original transcript, for example th*
    # These are ignored by ASR
    # Remove words that end with asterisks (e.g., th*)
    text = re.sub(r'\S*\*', '', text)

    # Remove all punctuation
    text = text.translate(punctuation_remover)

    # Clean up excess spaces in the original transcript or resulting from above operations
    text = re.sub(r'\s+', ' ', text)

    # Convert number representations, e.g., "thirteen" to "13"
    # This is imperfect (does not know when "one" is a pronoun vs. a number)
    # But we apply the same normalization to both samples, so works fine for Word Error Rate
    try:
        normalized_text = t2d.convert(text)
    except:
        print(text)
    return normalized_text.strip().lower()

In [30]:
wer = load("wer")
predictions = [normalize_transcript(d["text"]) for d in results]
references = [normalize_transcript(text) for text in dss["cslu"]["sentence"]]
wer_score = wer.compute(predictions=predictions, references=references)
print(wer_score) # 0.20958586984480837 on 07-08-25

0.20958586984480837


In [6]:
normalizer = BasicTextNormalizer()

def normalize_transcript(text):
    # The original transcript has annotations, for example a pause is <pau>
    # Remove tags in angle brackets
    text = re.sub(r'<[^>]*>', '', text)
    
    # These are "false starts" in the original transcript, for example th*
    # These are ignored by ASR
    # Remove words that end with asterisks (e.g., th*)
    text = re.sub(r'\S*\*', '', text)

    # Apply Whisper's English normalizer
    normalized_text = normalizer(text)

    return normalized_text

In [7]:
def weighted_wer(ref: list[str], pred: list[str]):
    # Normalize both predictions and references
    pred_normalized = [normalize_transcript(text) for text in pred]
    label_normalized = [normalize_transcript(text) for text in ref]
    
    total_errors = 0
    total_words = 0
    
    for pred_text, ref_text in zip(pred_normalized, label_normalized):
        ref_words = ref_text.split()
            
        # Compute WER for this sample
        sample_wer = jiwer.wer(ref_text, pred_text)
        
        # Accumulate weighted errors
        sample_errors = sample_wer * len(ref_words)
        total_errors += sample_errors
        total_words += len(ref_words)
    
    weighted_wer = total_errors / total_words if total_words > 0 else 0.0
    
    return {"wer": weighted_wer}

predictions = [d["text"] for d in results]
references = dss["cslu"]["sentence"]
wer_score = weighted_wer(references, predictions)
print(wer_score)

{'wer': 0.2082938074963048}
